In [ ]:
!pip install groq gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 4.1 MB/s eta 0:00:00


In [ ]:
import gradio as gr
from groq import Groq
import os

# Global variable to store the client
groq_client = None
api_key_verified = False

def verify_api_key(api_key):
    """Verify the Groq API key"""
    global groq_client, api_key_verified

    if not api_key or api_key.strip() == "":
        return "❌ Please enter an API key", False

    try:
        # Try to initialize the client and make a test call
        test_client = Groq(api_key=api_key.strip())

        # Make a simple test call to verify the key works
        test_response = test_client.chat.completions.create(
            messages=[{"role": "user", "content": "Hi"}],
            model="llama-3.1-8b-instant",
            max_tokens=10
        )

        # If we get here, the key is valid
        groq_client = test_client
        api_key_verified = True
        return "✅ API Key verified successfully! You can now chat.", True

    except Exception as e:
        groq_client = None
        api_key_verified = False
        return f"❌ API Key verification failed: {str(e)}", False

def is_skincare_related(message):
    """Check if the message is related to skincare"""
    skincare_keywords = [
        'skin', 'skincare', 'acne', 'pimple', 'wrinkle', 'moisturizer',
        'cleanser', 'serum', 'sunscreen', 'spf', 'face', 'facial',
        'cream', 'lotion', 'toner', 'exfoliate', 'mask', 'pore',
        'oil', 'dry', 'oily', 'sensitive', 'rosacea', 'eczema',
        'dark spot', 'pigmentation', 'aging', 'anti-aging', 'retinol',
        'hyaluronic', 'vitamin c', 'niacinamide', 'routine', 'beauty',
        'cosmetic', 'derma', 'complexion', 'blemish', 'blackhead',
        'whitehead', 'treatment', 'hydrat', 'glow', 'radiant'
    ]

    message_lower = message.lower()
    return any(keyword in message_lower for keyword in skincare_keywords)

def chat(message, history):
    """Main chat function"""
    global groq_client, api_key_verified

    # Check if API key is verified
    if not api_key_verified or groq_client is None:
        return "⚠️ Please verify your API key first using the button above."

    # Check if message is skincare-related
    if not is_skincare_related(message):
        return "I'm a specialized skincare chatbot and can only answer questions related to skincare, skin health, beauty routines, and skin products. Please ask me something about skincare! 😊"

    try:
        # Build the conversation history for the API
        messages = [
            {
                "role": "system",
                "content": "You are a knowledgeable and friendly skincare expert assistant. You only answer questions related to skincare, skin health, beauty routines, skin products, and dermatology. Be helpful, informative, and provide practical advice. If asked about anything unrelated to skincare, politely decline and remind the user you only discuss skincare topics."
            }
        ]

        # Add conversation history
        for user_msg, assistant_msg in history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})

        # Add current message
        messages.append({"role": "user", "content": message})

        # Get response from Groq
        response = groq_client.chat.completions.create(
            messages=messages,
            model="llama-3.1-8b-instant",
            max_tokens=1024,
            temperature=0.7
        )

        return response.choices[0].message.content

    except Exception as e:
        return f"❌ Error: {str(e)}"

# Create Gradio interface
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🧴 Skincare Expert Chatbot")
    gr.Markdown("Your personal AI skincare consultant powered by Groq")

    with gr.Row():
        api_key_input = gr.Textbox(
            label="Groq API Key",
            type="password",
            placeholder="Enter your Groq API key here...",
            scale=3
        )
        verify_btn = gr.Button("🔑 Verify API Key", scale=1, variant="primary")

    api_status = gr.Textbox(
        label="API Key Status",
        interactive=False,
        value="⏳ Waiting for API key verification..."
    )

    chatbot = gr.Chatbot(
        label="Chat",
        height=400,
        show_copy_button=True
    )

    msg = gr.Textbox(
        label="Your Message",
        placeholder="Ask me anything about skincare...",
        lines=2
    )

    with gr.Row():
        submit_btn = gr.Button("Send", variant="primary")
        clear_btn = gr.Button("Clear Chat")

    gr.Markdown("""
    ### 💡 Example Questions:
    - What's a good skincare routine for oily skin?
    - How do I treat acne?
    - What's the difference between retinol and retinoids?
    - Should I use sunscreen every day?
    - How can I reduce dark circles under my eyes?
    """)

    # Set up event handlers
    verify_btn.click(
        fn=verify_api_key,
        inputs=[api_key_input],
        outputs=[api_status]
    )

    submit_btn.click(
        fn=chat,
        inputs=[msg, chatbot],
        outputs=[chatbot]
    ).then(
        lambda: "",
        outputs=[msg]
    )

    msg.submit(
        fn=chat,
        inputs=[msg, chatbot],
        outputs=[chatbot]
    ).then(
        lambda: "",
        outputs=[msg]
    )

    clear_btn.click(
        fn=lambda: None,
        inputs=None,
        outputs=[chatbot]
    )

# Launch the app
if __name__ == "__main__":
    demo.launch(debug=True)

In [ ]:
import gradio as gr
from groq import Groq
import os

# Global variable to store the client
groq_client = None
api_key_verified = False

def verify_api_key(api_key):
    """Verify the Groq API key"""
    global groq_client, api_key_verified

    if not api_key or api_key.strip() == "":
        return "❌ Please enter an API key", False

    try:
        # Try to initialize the client and make a test call
        test_client = Groq(api_key=api_key.strip())

        # Make a simple test call to verify the key works
        test_response = test_client.chat.completions.create(
            messages=[{"role": "user", "content": "Hi"}],
            model="llama-3.1-8b-instant",
            max_tokens=10
        )

        # If we get here, the key is valid
        groq_client = test_client
        api_key_verified = True
        return "✅ API Key verified successfully! You can now chat.", True

    except Exception as e:
        groq_client = None
        api_key_verified = False
        return f"❌ API Key verification failed: {str(e)}", False

def is_skincare_related(message):
    """Check if the message is related to skincare"""
    skincare_keywords = [
        'skin', 'skincare', 'acne', 'pimple', 'wrinkle', 'moisturizer',
        'cleanser', 'serum', 'sunscreen', 'spf', 'face', 'facial',
        'cream', 'lotion', 'toner', 'exfoliate', 'mask', 'pore',
        'oil', 'dry', 'oily', 'sensitive', 'rosacea', 'eczema',
        'dark spot', 'pigmentation', 'aging', 'anti-aging', 'retinol',
        'hyaluronic', 'vitamin c', 'niacinamide', 'routine', 'beauty',
        'cosmetic', 'derma', 'complexion', 'blemish', 'blackhead',
        'whitehead', 'treatment', 'hydrat', 'glow', 'radiant'
    ]

    message_lower = message.lower()
    return any(keyword in message_lower for keyword in skincare_keywords)

def chat(message, history):
    """Main chat function"""
    global groq_client, api_key_verified

    # Check if API key is verified
    if not api_key_verified or groq_client is None:
        history.append([message, "⚠️ Please verify your API key first using the button above."])
        return history

    # Check if message is skincare-related
    if not is_skincare_related(message):
        history.append([message, "I'm a specialized skincare chatbot and can only answer questions related to skincare, skin health, beauty routines, and skin products. Please ask me something about skincare! 😊"])
        return history

    try:
        # Build the conversation history for the API
        messages = [
            {
                "role": "system",
                "content": "You are a knowledgeable and friendly skincare expert assistant. You only answer questions related to skincare, skin health, beauty routines, skin products, and dermatology. Be helpful, informative, and provide practical advice. If asked about anything unrelated to skincare, politely decline and remind the user you only discuss skincare topics."
            }
        ]

        # Add conversation history
        for user_msg, assistant_msg in history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})

        # Add current message
        messages.append({"role": "user", "content": message})

        # Get response from Groq
        response = groq_client.chat.completions.create(
            messages=messages,
            model="llama-3.1-8b-instant",
            max_tokens=1024,
            temperature=0.7
        )

        bot_response = response.choices[0].message.content
        history.append([message, bot_response])
        return history

    except Exception as e:
        history.append([message, f"❌ Error: {str(e)}"])
        return history

# Create Gradio interface
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🧴 Skincare Expert Chatbot")
    gr.Markdown("Your personal AI skincare consultant powered by Groq")

    with gr.Row():
        api_key_input = gr.Textbox(
            label="Groq API Key",
            type="password",
            placeholder="Enter your Groq API key here...",
            scale=3
        )
        verify_btn = gr.Button("🔑 Verify API Key", scale=1, variant="primary")

    api_status = gr.Textbox(
        label="API Key Status",
        interactive=False,
        value="⏳ Waiting for API key verification..."
    )

    chatbot = gr.Chatbot(
        label="Chat",
        height=400,
        show_copy_button=True
    )

    msg = gr.Textbox(
        label="Your Message",
        placeholder="Ask me anything about skincare...",
        lines=2
    )

    with gr.Row():
        submit_btn = gr.Button("Send", variant="primary")
        clear_btn = gr.Button("Clear Chat")

    gr.Markdown("""
    ### 💡 Example Questions:
    - What's a good skincare routine for oily skin?
    - How do I treat acne?
    - What's the difference between retinol and retinoids?
    - Should I use sunscreen every day?
    - How can I reduce dark circles under my eyes?
    """)

    # Set up event handlers
    verify_btn.click(
        fn=verify_api_key,
        inputs=[api_key_input],
        outputs=[api_status]
    )

    submit_btn.click(
        fn=chat,
        inputs=[msg, chatbot],
        outputs=[chatbot]
    ).then(
        lambda: "",
        outputs=[msg]
    )

    msg.submit(
        fn=chat,
        inputs=[msg, chatbot],
        outputs=[chatbot]
    ).then(
        lambda: "",
        outputs=[msg]
    )

    clear_btn.click(
        fn=lambda: [],
        inputs=None,
        outputs=[chatbot]
    )

# Launch the app
if __name__ == "__main__":
    demo.launch(debug=True)

/tmp/ipython-input-2449712972.py:101: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-2449712972.py:120: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(
/tmp/ipython-input-2449712972.py:120: DeprecationWarning: The 'show_copy_button' parameter will be removed in Gradio 6.0. You will need to use 'buttons=["copy"]' instead.
  chatbot = gr.Chatbot(
/tmp/ipython-input-2449712972.py:120: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8aef3e215b52aa917f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
